In [1]:
print('hello')

hello


In [2]:
!which python

/opt/conda/bin/python


In [5]:
!which java

/usr/bin/java


In [3]:
!python --version

Python 3.11.6


In [4]:
!java -version

openjdk version "17.0.8.1" 2023-08-24
OpenJDK Runtime Environment (build 17.0.8.1+1-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 17.0.8.1+1-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [6]:
pip show pyspark

Name: pyspark
Version: 3.5.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/spark/python
Requires: py4j
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
#SPARK_HOME
import os
os.environ.get('SPARK_HOME')

'/usr/local/spark'

In [8]:
#JAVA_HOME
os.environ.get('JAVA_HOME')


In [9]:
#PYTHONPATH
os.environ.get('PYTHONPATH')

'/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip:/usr/local/spark/python:'

In [10]:
import pyspark

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark example1').getOrCreate() #chaining
#SparkContext.SparkSession

In [12]:
spark

In [13]:
spark.stop()

In [14]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate() #chaining

In [15]:
spark

In [18]:
data = [('Alice',1), ('Bob', 2), ('Charlie',3) ]
type(data)

list

In [22]:
#DataFrame 객체(분산객체)를 생성 <> 판다스의 데이터프레임이 아님.
data1 = spark.createDataFrame( data, ['Name','Value'])
data1[1][1]

Column<'Value[1]'>

In [23]:
data1.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [37]:
data1.filter(data1.Name == 'Bob').show()

+----+-----+
|Name|Value|
+----+-----+
| Bob|    2|
+----+-----+



In [38]:
data1.filter(data1.Value > 2).show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



In [39]:
data1.createOrReplaceTempView('people')

In [41]:
spark.sql('select * from people').show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [42]:
spark.sql( 'select * from people where Name="Bob"' ).show()

+----+-----+
|Name|Value|
+----+-----+
| Bob|    2|
+----+-----+



In [43]:
spark.sql( 'select * from people where Value>2' ).show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



# RDD 객체 생성

In [24]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [26]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5]) #직접 생성
rdd

ParallelCollectionRDD[22] at readRDDFromFile at PythonRDD.scala:289

In [27]:
data1

DataFrame[Name: string, Value: bigint]

In [29]:
rdd.take(5) #rdd 객체를 출력하는 함수 - n개를 지정

[1, 2, 3, 4, 5]

In [30]:
#map 연산 : rdd 값으로 연산
squared_rdd = rdd.map(lambda x:x*x)
squared_rdd

PythonRDD[25] at RDD at PythonRDD.scala:53

In [32]:
rdd.take(3)

[1, 2, 3]

In [31]:
squared_rdd.take(3)

[1, 4, 9]

In [33]:
squared_rdd.collect()

[1, 4, 9, 16, 25]

# MLlib

In [44]:
from pyspark.ml.regression import LinearRegression

In [45]:
import numpy as np

In [46]:
from pyspark.ml.feature import VectorAssembler

In [48]:
data_age = [('Alice',25), ('Bob', 30), ('Charlie',33) ]
data2 = spark.createDataFrame( data_age, ['Name','Age'])
data2

DataFrame[Name: string, Age: bigint]

In [49]:
assembler = VectorAssembler(inputCols = ['Age'], outputCol='features')
vector_df = assembler.transform(data2)
vector_df

DataFrame[Name: string, Age: bigint, features: vector]

In [51]:
lr = LinearRegression(featuresCol='features', labelCol='Age')
model = lr.fit(vector_df)

In [52]:
pred = model.transform(vector_df)
pred

DataFrame[Name: string, Age: bigint, features: vector, prediction: double]

In [53]:
pred.show()

+-------+---+--------+-----------------+
|   Name|Age|features|       prediction|
+-------+---+--------+-----------------+
|  Alice| 25|  [25.0]|24.99999999999993|
|    Bob| 30|  [30.0]|30.00000000000001|
|Charlie| 33|  [33.0]|33.00000000000006|
+-------+---+--------+-----------------+



In [55]:
spark.stop()

# Streaming

In [54]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [56]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate() #chaining

In [ ]:
lines = spark.readStream.format('socket')\
            .option('host', 'localhost')\
            .option('port',9999)\
            .load()  #STREAMMING DATA

In [ ]:
aaaa bbbb cccc hello

In [ ]:
words = lines.select(explode(split(lines.value, ' ' )).alias('word')

In [57]:
spark.stop()